# Fleet Clustering

### Tim Hochberg, 2019-01-16

## Longliner Edition

We cluster vessel using HDBSCAN and a custom metric to derive fleets
that are related in the sense that they spend a lot of time in the same
location while at sea.

## See Also

* Other notebooks in https://github.com/GlobalFishingWatch/fleet-clustering for 
examples of clustering Squid Jiggers, etc.
* This workspace that Nate put together: https://globalfishingwatch.org/map/workspace/udw-v2-85ff8c4f-fbfe-4126-b067-4d94cdd2b737



## Open Questions

### Fleet Coherence Time

One thing this current implementation doesn't take into account is 
the coherence time of a fleet. A vessel might be part of one fleet 
this season, but move to another fleet the next season. A way to
deal with this is to group fleets over shorter time periods (6 months
for instance) and then match fleets across groupings by seeing what
previous fleets have the largest overlap with the current set of
fleets.

In [1]:
from __future__ import print_function
from __future__ import division
from collections import Counter, OrderedDict
import datetime as dt
import hdbscan
import logging
import matplotlib.pyplot as plt
import matplotlib.animation as mpl_animation
import numpy as np
import pandas as pd
from skimage import color
from IPython.display import HTML
from fleet_clustering import bq
from fleet_clustering import filters
from fleet_clustering import distances
from fleet_clustering import animation

## Load AIS Clustering Data

Load the AIS data that we use for clustering. Note that it onlyu includes vessels away
from shores so as to exclude clustering on ports

In [2]:
all_by_date = bq.load_ais_by_date('drifting_longlines', dt.date(2017, 1, 1), dt.date(2017, 12, 31),
                                 fishing_only=False, min_km_from_shore=0)    
pruned_by_date = {k : filters.remove_near_shore(10,
                            filters.remove_chinese_coast(v)) for (k, v) in all_by_date.items()}
valid_ssvid = sorted(filters.find_valid_ssvid(pruned_by_date))

2017-01-01


//anaconda/envs/ml2/lib/python2.7/site-packages/google/auth/_default.py:66: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK. We recommend that most server applications use service accounts instead. If your application continues to use end user credentials from Cloud SDK, you might receive a "quota exceeded" or "API not enabled" error. For more information about service accounts, see https://cloud.google.com/docs/authentication/.
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


## Create Distance Metrics

Create an array of distance metrics. The details are still evolving, but in general
we want to deal with two things.  Days on which a boat is missing and days where the
boat is away from the fleet.

* Distances to/from a boat on days when it is missing are represented by $\infty$ in 
  the distance matrix. HDBSCAN ignores these values.
* Only the closest N days are kept for each boat pair, allowing boats to leave the fleet
  for up to half the year without penalty.
  
In addition, distances have a floor of 1 km to prevent overclustering when boats tie up
up together, etc.

In [3]:
import imp; imp.reload(distances)
C = distances.create_composite_lonlat_array(pruned_by_date, valid_ssvid)
dists = distances.compute_distances_4(C, gamma=2)

## Load Carrier Data

In [4]:
carriers_by_date = bq.load_carriers_by_year(2017, 2018)
pruned_carriers_by_date = {k : filters.remove_chinese_coast(v) for (k, v) in carriers_by_date.items()}
query = """
               SELECT CAST(mmsi AS STRING) FROM
               `world-fishing-827.vessel_database.all_vessels_20190102`
               WHERE  iscarriervessel AND confidence = 3
        """
valid_carrier_ssvid_df = pd.read_gbq(query, dialect='standard', project_id='world-fishing-827')
valid_carrier_ssvid = valid_carrier_ssvid_df.f0_
valid_carrier_ssvid_set = set(valid_carrier_ssvid)

## Load Encounters Data And Country Codes

This is used to filter the carrier vessels down to only those
that meet with longliners and to add iso3 labels to outputs

In [23]:
encounters = bq.load_carriers(2017, 2017)

In [ ]:
query = """
SELECT code, iso3 FROM `world-fishing-827.gfw_research.country_codes`"""
country_codes_df = pd.read_gbq(query, dialect='standard', project_id='world-fishing-827')
iso3_map = {x.code : x.iso3 for x in country_codes_df.itertuples()}

## Fit the Clusterer

This is pretty straightforward -- all the complicated stuff is
embedded in the matrix computations. Fleet size can be tweaked
using `min_cluster_size` and `min_sample_size`.

In [79]:
clusterer = hdbscan.HDBSCAN(metric='precomputed', 
                            min_cluster_size=10,
                           )
clusterer.fit(dists)

HDBSCAN(algorithm='best', allow_single_cluster=False, alpha=1.0,
    approx_min_span_tree=True, cluster_selection_method='eom',
    core_dist_n_jobs=4, gen_min_span_tree=False, leaf_size=40,
    match_reference_implementation=False, memory=Memory(cachedir=None),
    metric='precomputed', min_cluster_size=10, min_samples=None, p=None,
    prediction_data=False)

## Set up Fleets

Set up the fleets for viewing.

In [80]:
all_fleet_ssvid_set = set([s for (s, f) in zip(valid_ssvid, clusterer.labels_) if f >= 0])
valid_ssvid_set = set(valid_ssvid)
all_longline_reefer_ssvid_set = set()
for x in encounters.itertuples():
    if x.ssvid_1 in all_fleet_ssvid_set and x.ssvid_2 in valid_carrier_ssvid_set:
        all_longline_reefer_ssvid_set.add(x.ssvid_2)
    if x.ssvid_2 in all_fleet_ssvid_set and x.ssvid_1 in valid_carrier_ssvid_set:
        all_longline_reefer_ssvid_set.add(x.ssvid_1)
all_longline_reefer_ssvid = sorted(all_longline_reefer_ssvid_set)

valid_ssvid_set = set(valid_ssvid)
carrier_ids = [x for x in all_longline_reefer_ssvid if x not in valid_ssvid_set]
joint_ssvid = valid_ssvid + sorted(carrier_ids) 
labels = list(clusterer.labels_) + [max(clusterer.labels_) + 1] * len(carrier_ids) 

In [81]:
counts = []
skip = [2, 9, 12, 4] 
for i in range(max(labels) + 1):
    if i in skip:
        counts.append(0)
    else:
        counts.append((np.array(labels) == i).sum())
        
fleet_ids = [x for x in np.argsort(counts)[::-1] if counts[x] > 0]
fleet_ids_without_carriers = [x for x in fleet_ids if x != max(labels)]

print(len(fleet_ids), "fleets")
fleets = OrderedDict()
n_hues = int(np.ceil(len(fleet_ids) / 4.0))
used = set()
for i, fid in enumerate(fleet_ids_without_carriers):
    b = (i // (2 * n_hues)) % 2
    c = (i // 2)% n_hues
    d = i  % 2
    symbol = 'o^'[d]
    assert (b, c, d) not in used, (i, b, c, d)
    used.add((b, c, d))
    sat = 1
    val = 1
    hue = c / float(n_hues)
    assert 0 <= hue < 1, hue
    [[clr]] = color.hsv2rgb([[(hue, sat, val)]])
    fg = [(0, 0, 0), clr][b]
    bg = [clr, (1, 1, 1)][b]
    w = [1, 2][b]
    sz = [7, 7][b]
    fleets[fid] = (symbol, tuple(fg), tuple(bg), sz, w,  str(i + 1))
# fleets[max(labels)] = ('1', 'k', 'k', 8, 2, 'Carrier Vessel')

34 fleets


## Create Animations

In [82]:
anim = animation.make_anim(joint_ssvid, 
                           labels, 
                           all_by_date, 
                           interval=10,
                           fleets=fleets, 
                           show_ungrouped=True,
                           alpha=1,
                           legend_cols=8,
                           ungrouped_legend="Ungrouped")
HTML(anim.to_html5_video())

In [83]:
anim = animation.make_anim(joint_ssvid, 
                           labels, 
                           all_by_date, 
                           interval=1,
                           fleets=fleets, 
                           show_ungrouped=True,
                           alpha=1,
                           legend_cols=8,
                           ungrouped_legend="Ungrouped")
Writer = mpl_animation.writers['ffmpeg']
writer = Writer(fps=8, metadata=dict(artist='Me'), bitrate=1800)
anim.save('fleet_longlines.mp4', writer=writer)

## List Fleet Composition

In [84]:
for fid, v in fleets.items():
    label = v[-1]
    mask = (fid == np.array(labels))
    ssvids = np.array(joint_ssvid)[mask]
    mids = [x[:3] for x in ssvids]
    countries = [iso3_map.get(float(x), x) for x in mids]
    c = Counter(countries)
    print('Fleet: {} ({})'.format(label, fid), label)
    for country, count in c.most_common():
        print('\t', country, ':', count)

Fleet: 1 (26) 1
	 JPN : 128
	 KOR : 1
Fleet: 2 (34) 2
	 CHN : 68
	 KOR : 16
	 FJI : 13
Fleet: 3 (6) 3
	 USA : 49
	 CAN : 45
	 PAN : 1
Fleet: 4 (32) 4
	 TWN : 41
	 SYC : 8
	 TZA : 1
Fleet: 5 (30) 5
	 CHN : 22
	 TWN : 12
	 FSM : 6
	 MHL : 4
	 200 : 1
Fleet: 6 (13) 6
	 TWN : 23
	 CHN : 19
	 KOR : 1
	 JPN : 1
Fleet: 7 (23) 7
	 CHN : 21
	 TWN : 7
	 SYC : 5
	 CYP : 1
Fleet: 8 (11) 8
	 ESP : 28
	 MLT : 3
	 ITA : 1
Fleet: 9 (21) 9
	 USA : 32
Fleet: 10 (29) 10
	 CHN : 13
	 TWN : 9
	 VUT : 7
Fleet: 11 (24) 11
	 USA : 11
	 VUT : 7
	 TWN : 4
	 KIR : 1
	 COK : 1
	 MDV : 1
Fleet: 12 (3) 12
	 IND : 21
	 420 : 1
Fleet: 13 (18) 13
	 JPN : 21
	 TWN : 1
Fleet: 14 (31) 14
	 TWN : 16
	 MYS : 6
Fleet: 15 (15) 15
	 ZAF : 21
Fleet: 16 (5) 16
	 AUS : 21
Fleet: 17 (33) 17
	 FJI : 14
	 CHN : 5
	 452 : 1
	 600 : 1
Fleet: 18 (25) 18
	 TWN : 20
	 FJI : 1
Fleet: 19 (19) 19
	 JPN : 15
	 KOR : 2
	 433 : 1
	 CHN : 1
Fleet: 20 (22) 20
	 REU : 18
	 FRA : 1
Fleet: 21 (1) 21
	 BRA : 9
	 719 : 3
	 711 : 2
	 718 : 1
	 713 : 

## Isolate Selected Fleets

Look at a few fleets that move around a lot.

In [91]:
fleets_selected = {k : v for (k, v) in fleets.items() 
                    if v[-1] in ('13', '19', '28', '32')}
anim = animation.make_anim(joint_ssvid, 
                           labels, 
                           all_by_date, 
                           interval=10,
                           fleets=fleets_selected, 
                           show_ungrouped=True,
                           alpha=1,
                           legend_cols=8,
                           ungrouped_legend="Ungrouped")
HTML(anim.to_html5_video())

In [92]:
anim = animation.make_anim(joint_ssvid, 
                           labels, 
                           all_by_date, 
                           interval=1,
                           fleets=fleets_selected, 
                           show_ungrouped=True,
                           alpha=1,
                           legend_cols=8,
                           ungrouped_legend="Ungrouped")
Writer = mpl_animation.writers['ffmpeg']
writer = Writer(fps=8, metadata=dict(artist='Me'), bitrate=1800)
anim.save('fleet_longlines_selected.mp4', writer=writer)

In [95]:
for fid in fleets_selected:
    print('fleet', fleets_selected[fid][-1])
    mask = (fid == labels)
    mmsi = np.array(joint_ssvid)[mask]
    print('\t', mmsi)

fleet 32
	 [u'416039500' u'416785000' u'576596000' u'576609000' u'576633000'
 u'576635000' u'576636000' u'576649000' u'576679000' u'576682000'
 u'576690000' u'576726000' u'576975000']
fleet 28
	 [u'431316000' u'431620000' u'431682000' u'431701010' u'431701040'
 u'431701650' u'431813000' u'432045000' u'432306000' u'432367000'
 u'432458000' u'432605000' u'432856000' u'432967000' u'440594000']
fleet 13
	 [u'416772000' u'431047000' u'431067000' u'431116000' u'431200650'
 u'431253000' u'431448000' u'431700560' u'431700970' u'431700980'
 u'431702230' u'432282000' u'432353000' u'432405000' u'432475000'
 u'432568000' u'432848000' u'432878000' u'432894000' u'432918000'
 u'432955000' u'432987000']
fleet 19
	 [u'412123457' u'431200030' u'431200730' u'431200740' u'431300000'
 u'431457000' u'431600010' u'431602710' u'431700130' u'431702580'
 u'431725000' u'431750000' u'431791000' u'432408000' u'432426000'
 u'432444000' u'433226000' u'440926000' u'441645000']


## Repeat for Coarser Clustering


In [68]:
clusterer = hdbscan.HDBSCAN(metric='precomputed', 
                            min_cluster_size=14,
                           )
clusterer.fit(dists)

HDBSCAN(algorithm='best', allow_single_cluster=False, alpha=1.0,
    approx_min_span_tree=True, cluster_selection_method='eom',
    core_dist_n_jobs=4, gen_min_span_tree=False, leaf_size=40,
    match_reference_implementation=False, memory=Memory(cachedir=None),
    metric='precomputed', min_cluster_size=14, min_samples=None, p=None,
    prediction_data=False)

In [69]:
all_fleet_ssvid_set = set([s for (s, f) in zip(valid_ssvid, clusterer.labels_) if f >= 0])
valid_ssvid_set = set(valid_ssvid)
all_longline_reefer_ssvid_set = set()
for x in encounters.itertuples():
    if x.ssvid_1 in all_fleet_ssvid_set and x.ssvid_2 in valid_carrier_ssvid_set:
        all_longline_reefer_ssvid_set.add(x.ssvid_2)
    if x.ssvid_2 in all_fleet_ssvid_set and x.ssvid_1 in valid_carrier_ssvid_set:
        all_longline_reefer_ssvid_set.add(x.ssvid_1)
all_longline_reefer_ssvid = sorted(all_longline_reefer_ssvid_set)

valid_ssvid_set = set(valid_ssvid)
carrier_ids = [x for x in all_longline_reefer_ssvid if x not in valid_ssvid_set]
joint_ssvid = valid_ssvid + sorted(carrier_ids) 
labels = list(clusterer.labels_) + [max(clusterer.labels_) + 1] * len(carrier_ids) 

In [72]:
counts = []
skip = []
for i in range(max(labels) + 1):
    if i in skip:
        counts.append(0)
    else:
        counts.append((np.array(labels) == i).sum())
        
fleet_ids = [x for x in np.argsort(counts)[::-1] if counts[x] > 0]
fleet_ids_without_carriers = [x for x in fleet_ids if x != max(labels)]

print(len(fleet_ids), "fleets")
fleets = OrderedDict()
n_hues = int(np.ceil(len(fleet_ids) / 4.0))
used = set()
for i, fid in enumerate(fleet_ids_without_carriers):
    b = (i // (2 * n_hues)) % 2
    c = (i // 2)% n_hues
    d = i  % 2
    symbol = 'o^'[d]
    assert (b, c, d) not in used, (i, b, c, d)
    used.add((b, c, d))
    sat = 1
    val = 1
    hue = c / float(n_hues)
    assert 0 <= hue < 1, hue
    [[clr]] = color.hsv2rgb([[(hue, sat, val)]])
    fg = [(0, 0, 0), clr][b]
    bg = [clr, (1, 1, 1)][b]
    w = [1, 2][b]
    sz = [7, 7][b]
    fleets[fid] = (symbol, tuple(fg), tuple(bg), sz, w,  str(i + 1))
fleets[max(labels)] = ('1', 'k', 'k', 8, 2, 'Carrier Vessel')

27 fleets


In [73]:
anim = animation.make_anim(joint_ssvid, 
                           labels, 
                           all_by_date, 
                           interval=10,
                           fleets=fleets, 
                           show_ungrouped=True,
                           alpha=1,
                           legend_cols=8,
                           ungrouped_legend="Ungrouped")
HTML(anim.to_html5_video())

In [74]:
anim = animation.make_anim(joint_ssvid, 
                           labels, 
                           all_by_date, 
                           interval=1,
                           fleets=fleets, 
                           show_ungrouped=True,
                           alpha=1,
                           legend_cols=8,
                           ungrouped_legend="Ungrouped")
Writer = mpl_animation.writers['ffmpeg']
writer = Writer(fps=8, metadata=dict(artist='Me'), bitrate=1800)
anim.save('fleet_longlines_g14.mp4', writer=writer)

In [75]:
for fid, v in fleets.items():
    label = v[-1]
    mask = (fid == np.array(labels))
    ssvids = np.array(joint_ssvid)[mask]
    mids = [x[:3] for x in ssvids]
    countries = [iso3_map.get(float(x), x) for x in mids]
    c = Counter(countries)
    print('Fleet: {} ({})'.format(label, fid), label)
    for country, count in c.most_common():
        print('\t', country, ':', count)

Fleet: 1 (6) 1
	 ESP : 73
	 PRT : 27
	 MLT : 27
	 GRC : 20
	 CYP : 12
	 ITA : 6
	 FRA : 1
Fleet: 2 (17) 2
	 CHN : 83
	 FJI : 34
	 KOR : 15
	 452 : 2
	 VUT : 2
	 600 : 1
	 AUS : 1
Fleet: 3 (16) 3
	 JPN : 122
	 KOR : 1
Fleet: 4 (3) 4
	 CHN : 95
	 415 : 4
	 DEU : 3
	 KIR : 1
	 556 : 1
	 KOR : 1
Fleet: 5 (4) 5
	 USA : 46
	 CAN : 43
Fleet: 6 (25) 6
	 TWN : 41
	 SYC : 8
	 TZA : 1
Fleet: 7 (19) 7
	 KOR : 48
	 TWN : 1
Fleet: 8 (13) 8
	 USA : 48
Fleet: 9 (1) 9
	 BRA : 12
	 ESP : 10
	 TWN : 10
	 719 : 3
	 711 : 2
	 PRT : 2
	 713 : 1
	 718 : 1
	 430 : 1
	 GRL : 1
Fleet: 10 (23) 10
	 CHN : 24
	 TWN : 8
	 FSM : 7
	 MHL : 4
Fleet: 11 (8) 11
	 TWN : 17
	 CHN : 17
Fleet: 12 (0) 12
	 BHR : 21
	 CHN : 2
	 QAT : 1
	 FRA : 1
	 OMN : 1
	 TZA : 1
	 IND : 1
Fleet: 13 (20) 13
	 CHN : 14
	 VUT : 7
	 TWN : 4
Fleet: 14 (14) 14
	 CHN : 22
Fleet: 15 (7) 15
	 ZAF : 22
Fleet: 16 (11) 16
	 JPN : 21
Fleet: 17 (21) 17
	 CHN : 16
	 TWN : 4
	 SYC : 1
Fleet: 18 (22) 18
	 REU : 16
	 MUS : 4
	 FRA : 1
Fleet: 19 (5) 19
	 AUS